Testing Gemini models using the Google AI Studio + Gemini credits (300 dollars).

In [1]:
from dotenv import load_dotenv
import os

# Load environment variables from the .env file
load_dotenv()

# Read the API key
API_KEY = os.getenv("GOOGLE_API_KEY")

ModuleNotFoundError: No module named 'dotenv'

In [10]:
from google import genai
from google.genai import types
import os
client = genai.Client(api_key=API_KEY)

inputTest = """"
This is the attributes extracted from the database for you to enrich the properties

name: "Guarulhos Airport"
iata: "GRU"
country: "Brazil"
city: "Sao Paulo"
"""

promptTest = """"
# Identity

You are an assistant that enrichs entities from a database for analytics by identifying the corresponding real-world entity and adding new trustworthy factual attributes. These attributes will be transformed into property:value pairs in a property-graph database.

# Instructions

1. From the input node (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.
2. Add the most factual information you can confidently verify for the entity class. Prioritize core, verifiable facts (dates, identifiers, official names, locations, standardized codes, market share, estimated revenue).
3. Use consistent property names and information across entities of the same class. Try to make entities share the most amount of high-value information.

# Output format

Out a CSV with header `property,value`.
These will be related only to properties of nodes.
If no facts can be verified, output only the header row.
"""

folder_path = "../prompts/"

files = os.listdir(folder_path)
texts = []

for filename in files:
    if filename.endswith(".txt"):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()
            texts.append(content)

modelName = "gemini-2.5-pro"

# Run for gemini3 also later

for index,prompt in enumerate(texts):

    response = client.models.generate_content(
        model=modelName,
        config=types.GenerateContentConfig(
            system_instruction=prompt),
            contents=inputTest
    )

    output_path = "../outputs/"+f"{modelName}_{files[index]}"
    with open(output_path, "w", encoding="utf-8") as outputFile:
        outputFile.write(response.text)

In [21]:
from google import genai
from google.genai import types
import os
client = genai.Client(api_key=API_KEY)

inputTest = """"
This is the attributes extracted from the database for you to enrich the properties

name: "Guarulhos Airport"
iata: "GRU"
country: "Brazil"
city: "Sao Paulo"
"""

promptTest = """"
# Identity

You are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.

# Instructions
1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.
2. Find trustworthy and recent values for the following properties of this entitiy: iata, name, city. If you are not confident about the values, do not include them.
3. Do not include any other property and do not include information that already exists in the input.

# Output format

Out only CSV with header `property,value`. Do not output any other text or tags
These will be related only to properties of nodes.
If no facts can be verified, output only the header row.
"""


modelName = "gemini-2.5-pro"

# Run for gemini3 also later


response = client.models.generate_content(
    model=modelName,
    config=types.GenerateContentConfig(
        system_instruction=promptTest,tools=[]),
        contents=inputTest,
        

)
print(response.text)
print(response.usage_metadata)

property,value
cache_tokens_details=None cached_content_token_count=None candidates_token_count=3 candidates_tokens_details=None prompt_token_count=262 prompt_tokens_details=[ModalityTokenCount(
  modality=<MediaModality.TEXT: 'TEXT'>,
  token_count=262
)] thoughts_token_count=1194 tool_use_prompt_token_count=None tool_use_prompt_tokens_details=None total_token_count=1459 traffic_type=None


In [ ]:
from google import genai

client = genai.Client()
prompt = "The quick brown fox jumps over the lazy dog."

# Count tokens using the new client method.
total_tokens = client.models.count_tokens(
    model="gemini-2.0-flash", contents=prompt
)
print("total_tokens: ", total_tokens)
# ( e.g., total_tokens: 10 )

response = client.models.generate_content(
    model="gemini-2.0-flash", contents=prompt
)

# The usage_metadata provides detailed token counts.
print(response.usage_metadata)
# ( e.g., prompt_token_count: 11, candidates_token_count: 73, total_token_count: 84 )